In [71]:
import nltk
import time
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import r2_score


In [72]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # turn a doc into clean tokens
                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #stop_words = set(stopwords.words('english'))
                    
                    #tokens = [w for w in tokens if not w in stop_words]
                    
                    
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 2]
                    return tokens
                




               
                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words
                
                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
                    
                    
                def get_ngrams(words,n):
                    n_grams = ngrams(words, n)
                    
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams
    
            
                #convert to string
                idx=row["spans"]
                #ch="".join(x for x in idx if x)
                ch=' '.join(idx)
               
    
                
                
                n_grams=clean_doc(ch)
                #tokens=words_lemmatizer(n_grams)
                #print("nombre of tokens",len(n_grams))

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       ):
       
        self.wordlist = []
        whitelist=["to","on","for","up","below","short","long"]
        
        #stopwords=[]
       
        words = Counter()
        
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

       
            
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
                
                
        
                
        
        
        print(words.most_common())        
        
        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in (words.most_common()) if min_occurrences < v < max_occurences]
        
        
        
    def build_data_model(self):
        label_column = []
        Id_column=["ID"]
        label_column = ["label"]

        columns = Id_column + label_column + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                labels.append(current_label)
                
                current_row.append(current_id)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [73]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    whitelist = ["n't","not","below"]    

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40

                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #stop_words = set(stopwords.words('english'))
                    #tokens = [w for w in tokens if not w in stop_words]
                    
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 2]
                    return tokens


                

                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
                    
                def get_ngrams(words,n):
                    n_grams = ngrams(words, n)
                    
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams
    
            
                token=[]
                idx=row["spans"]
                ch=' '.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                n_grams=clean_doc(ch)
                #tokens=words_lemmatizer(n_grams)

                #print("list ngrams--------------\n",n_grams)
                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        
        whitelist=["to","on","for","up","below","short","long"]
        self.wordlist = []
        #stopwords=[]
        
        
        words = Counter()
        
        
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        
            
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
                
                
        
                
        
        
        print(words.most_common())        

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in (words.most_common()) if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        
        label_id = ["ID"]

        columns = label_id + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []
            if True:
                # add label
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                
                current_row.append(current_id)

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [74]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow]
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015]
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers]
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.]


In [75]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_Trialdata.json")

data_test.processed_data.head()
test_data=data_test.processed_data.copy()

In [76]:
data.do_process()

--- 1.165207862854004 seconds ---


In [77]:
data.processed_data.head(20)
tokens=data.processed_data["token_spans"]
taille=0
for token in tokens:
    taille+=len(token)
print(taille)



8314


In [78]:
data.processed_data.head(20)


,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watching, for, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, passengers, served]"
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],"[out, nflx]"
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[looking, for, strong, bounce, lunchtime, rall..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigued, with, the, technology, and, ..."
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]","[short, worked, puts]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought],[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP...","[absolute, garbage, still, stores, totally, em..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers],"[biggest, market, losers]"
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.],"[love, this, company, long, time]"


In [79]:

data.build_wordlist()
data.wordlist.sort()
def save_list(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
    
save_list(data.wordlist,"list_word_copie.txt")
len(data.wordlist)

[('for', 128), ('stocks', 82), ('long', 74), ('buy', 70), ('short', 68), ('today', 56), ('good', 46), ('nice', 44), ('new', 44), ('looking', 43), ('still', 43), ('stock', 43), ('market', 38), ('bullish', 37), ('day', 36), ('call', 34), ('top', 34), ('like', 31), ('week', 29), ('rally', 27), ('trade', 27), ('looks', 27), ('sell', 26), ('tsla', 25), ('buying', 24), ('aapl', 24), ('strong', 23), ('calls', 23), ('time', 22), ('could', 22), ('support', 22), ('back', 22), ('higher', 22), ('puts', 21), ('shorts', 21), ('close', 21), ('going', 20), ('high', 20), ('sold', 20), ('weak', 20), ('holding', 20), ('breakout', 20), ('growth', 19), ('overbought', 19), ('see', 19), ('get', 19), ('may', 19), ('one', 19), ('selling', 18), ('next', 18), ('great', 18), ('bought', 18), ('play', 18), ('run', 17), ('earnings', 17), ('setups', 17), ('morning', 17), ('coming', 16), ('unusual', 16), ('positive', 16), ('downgrade', 16), ('bull', 16), ('ready', 16), ('losers', 15), ('big', 15), ('apple', 15), ('mon

2135

In [80]:
#------------------Word2vec--------------------------------------------------
# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#list_tweet=get_tweets(data)
list_tweet=list(data.processed_data["token_spans"])
#sentences=review_sentences(list_tweet, remove_stopwords=True)
sentences=list_tweet

# Creating the model and setting values for the various parameters
num_features = 100  # Word vector dimensionality
min_word_count = 2 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "microblogs_word_vector"
model.save(model_name)
#model = gensim.models.Word2Vec.load(model_name)


2018-05-23 10:27:38,603 : INFO : collecting all words and their counts
2018-05-23 10:27:38,605 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-23 10:27:38,611 : INFO : collected 2214 word types from a corpus of 8314 raw words and 1700 sentences
2018-05-23 10:27:38,612 : INFO : Loading a fresh vocabulary
2018-05-23 10:27:38,617 : INFO : min_count=2 retains 1051 unique words (47% of original 2214, drops 1163)
2018-05-23 10:27:38,619 : INFO : min_count=2 leaves 7151 word corpus (86% of original 8314, drops 1163)
2018-05-23 10:27:38,624 : INFO : deleting the raw counts dictionary of 2214 items
2018-05-23 10:27:38,625 : INFO : sample=0.001 downsamples 73 most-common words
2018-05-23 10:27:38,626 : INFO : downsampling leaves estimated 6068 word corpus (84.9% of prior 7151)
2018-05-23 10:27:38,630 : INFO : estimated required memory for 1051 words and 100 dimensions: 1366300 bytes
2018-05-23 10:27:38,632 : INFO : resetting layer weights
2018-05-23 10:27:38,65

Training model....


2018-05-23 10:27:38,704 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-05-23 10:27:38,707 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-23 10:27:38,711 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-23 10:27:38,719 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-23 10:27:38,720 : INFO : EPOCH - 2 : training on 8314 raw words (6066 effective words) took 0.0s, 315296 effective words/s
2018-05-23 10:27:38,732 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-05-23 10:27:38,733 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-23 10:27:38,735 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-23 10:27:38,745 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-23 10:27:38,746 : INFO : EPOCH - 3 : training on 8314 raw words (6078 effective words) took 0.0s, 396488 effective words/s
2018-05-23

In [81]:
columns=list(map(lambda i: "word2vec_{0}".format(i), range(0, model.vector_size)))
import numpy as np
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

#transformer les donner de train en moyenne de vecteurs word2vec 
trainDataVecs = getAvgFeatureVecs(list_tweet, model, num_features)



Review 0 of 1700
Review 1000 of 1700


C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [82]:
rows=list(trainDataVecs)
data_model = pd.DataFrame(rows, columns=columns)
data_model.head()

,word2vec_0,word2vec_1,word2vec_2,word2vec_3,word2vec_4,word2vec_5,word2vec_6,word2vec_7,word2vec_8,word2vec_9,...,word2vec_90,word2vec_91,word2vec_92,word2vec_93,word2vec_94,word2vec_95,word2vec_96,word2vec_97,word2vec_98,word2vec_99
0,0.051253,-0.023325,0.043538,0.027733,-0.000572,0.072607,-0.005318,0.010735,-0.041563,0.062954,...,0.036176,-0.014585,-0.039435,-0.111412,0.062930,-0.070495,0.016721,0.044607,-0.010384,-0.033296
1,-0.053529,0.006522,-0.010614,-0.044481,0.023780,-0.007542,0.077665,-0.049258,-0.004307,0.020375,...,0.079903,-0.090251,-0.023795,0.085427,-0.039390,0.078473,-0.001703,-0.040614,0.043617,0.005226
2,-0.018119,0.035739,0.008004,-0.045642,0.007712,0.049206,-0.060233,0.079258,0.102539,0.133056,...,0.012059,-0.042455,-0.052324,-0.046249,0.015461,-0.107806,0.007347,0.092894,0.089371,-0.064510
3,-0.059184,0.022577,0.047356,0.111928,0.028518,0.068501,0.046202,0.037847,0.029007,0.121551,...,0.047720,-0.033584,0.097955,-0.010803,0.028270,-0.103141,0.007089,-0.005249,0.005323,0.025629
4,-0.013634,0.018760,-0.013325,0.128147,0.000789,0.068625,0.041031,0.072915,-0.028941,0.104396,...,0.058941,-0.074064,0.077727,-0.026842,0.043481,-0.046453,0.048603,0.030498,0.084867,-0.012579


In [83]:
bow, labels = data.build_data_model()
bow.head(20)


,ID,label,aal,aapl,aapls,absolute,abyss,accept,acct,accumulate,...,yoku,zero,zigzag,ziv,zmh,znga,zombie,zone,zuck,zuckerberg
0,719659409228451840,0.366,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,719904304207962112,0.638,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5329774,-0.494,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,719891468173844480,0.460,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20091246,0.403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5819749,0.000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,709741154393133056,-0.296,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,17892972,-0.546,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,709834259687710720,-0.438,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,708481442079068160,0.408,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
len(bow.columns)

2137

In [85]:
bow=pd.concat([bow,data_model],axis=1)
bow.head()

,ID,label,aal,aapl,aapls,absolute,abyss,accept,acct,accumulate,...,word2vec_90,word2vec_91,word2vec_92,word2vec_93,word2vec_94,word2vec_95,word2vec_96,word2vec_97,word2vec_98,word2vec_99
0,719659409228451840,0.366,0,0,0,0,0,0,0,0,...,0.036176,-0.014585,-0.039435,-0.111412,0.062930,-0.070495,0.016721,0.044607,-0.010384,-0.033296
1,719904304207962112,0.638,0,0,0,0,0,0,0,0,...,0.079903,-0.090251,-0.023795,0.085427,-0.039390,0.078473,-0.001703,-0.040614,0.043617,0.005226
2,5329774,-0.494,0,0,0,0,0,0,0,0,...,0.012059,-0.042455,-0.052324,-0.046249,0.015461,-0.107806,0.007347,0.092894,0.089371,-0.064510
3,719891468173844480,0.460,0,0,0,0,0,0,0,0,...,0.047720,-0.033584,0.097955,-0.010803,0.028270,-0.103141,0.007089,-0.005249,0.005323,0.025629
4,20091246,0.403,0,0,0,0,0,0,0,0,...,0.058941,-0.074064,0.077727,-0.026842,0.043481,-0.046453,0.048603,0.030498,0.084867,-0.012579


In [86]:
data_test.do_process()
data_test.processed_data.head(20)



,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[putting, little, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],"[short, some]"
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]"
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scaling, long, position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[its, time, sell, banks]"
5,$LMT,14073133,0.627,stocktwits,[Entering long],"[entering, long]"
6,$DNN,18479024,0.653,stocktwits,[picked some up],"[picked, some]"
7,$CRK,34147106,0.668,stocktwits,"[time to accumulate for a long position, far m...","[time, accumulate, for, long, position, far, m..."
8,$CRK,34147107,0.460,stocktwits,"[Looking for a strong bounce, Lunchtime rally ...","[looking, for, strong, bounce, lunchtime, rall..."
9,$CRK,34147108,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigued, with, the, technology, and, ..."


In [87]:
data_test.build_wordlist()
data_test.wordlist.sort()

print(len(data_test.wordlist))
bow_test= data_test.build_data_model()
bow_test.head(20)

[('short', 4), ('long', 3), ('putting', 2), ('little', 2), ('buying', 2), ('position', 2), ('time', 2), ('for', 2), ('opportunity', 1), ('scaling', 1), ('sell', 1), ('banks', 1), ('entering', 1), ('picked', 1), ('accumulate', 1), ('far', 1), ('upside', 1), ('downside', 1), ('looking', 1), ('strong', 1), ('bounce', 1), ('lunchtime', 1), ('rally', 1), ('coming', 1), ('intrigued', 1), ('technology', 1), ('growth', 1), ('potential', 1), ('worked', 1), ('puts', 1), ('biggest', 1), ('market', 1), ('losers', 1), ('goog', 1), ('googl', 1), ('would', 1), ('suck', 1), ('sbux', 1), ('dip', 1), ('below', 1), ('overbought', 1), ('dont', 1)]
42


,ID,accumulate,banks,below,biggest,bounce,buying,coming,dip,dont,...,scaling,sell,short,strong,suck,technology,time,upside,worked,would
0,5540055,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,10752226,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,10920221,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12971398,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,16142438,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
5,14073133,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,18479024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,34147106,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
8,34147107,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
9,34147108,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [88]:
bow_test.columns

Index(['ID', 'accumulate', 'banks', 'below', 'biggest', 'bounce', 'buying',
       'coming', 'dip', 'dont', 'downside', 'entering', 'far', 'for', 'goog',
       'googl', 'growth', 'intrigued', 'little', 'long', 'looking', 'losers',
       'lunchtime', 'market', 'opportunity', 'overbought', 'picked',
       'position', 'potential', 'puts', 'putting', 'rally', 'sbux', 'scaling',
       'sell', 'short', 'strong', 'suck', 'technology', 'time', 'upside',
       'worked', 'would'],
      dtype='object')

In [89]:
#data.wordlist.index("fit")
test_tweet=list(data_test.processed_data["token_spans"])
testDataVecs = getAvgFeatureVecs(test_tweet, model, num_features)
row_test=list(testDataVecs)
data_test_model = pd.DataFrame(row_test, columns=columns)
bow_test=pd.concat([bow_test,data_test_model],axis=1)

Review 0 of 16


C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [90]:

"""
#convert bow dataframe to numpy matrix

Y = bow.label.as_matrix()
X_drop = bow.drop({"label","ID"},axis=1)

X=X_drop.as_matrix()
"""

'\n#convert bow dataframe to numpy matrix\n\nY = bow.label.as_matrix()\nX_drop = bow.drop({"label","ID"},axis=1)\n\nX=X_drop.as_matrix()\n'

In [91]:
bow_columns=list(bow.columns)
for word in bow_columns:
    current_row=[]
    if (not word  in data_test.wordlist):
        # add label
        for idx in bow_test.index:
            current_row.append(0)
        bow_test[word]=current_row

    
bow_test.head(20)


,ID,accumulate,banks,below,biggest,bounce,buying,coming,dip,dont,...,yoku,zero,zigzag,ziv,zmh,znga,zombie,zone,zuck,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
#bow_test=pd.concat([bow_test,data_test_model],axis=1)
list(bow_test.columns).index("word2vec_90")
print((bow_test.columns))
#bow_test.sort_index(axis = 1, ascending = False)
#bow.sort_index(axis = 1, ascending = True)


Index(['ID', 'accumulate', 'banks', 'below', 'biggest', 'bounce', 'buying',
       'coming', 'dip', 'dont',
       ...
       'yoku', 'zero', 'zigzag', 'ziv', 'zmh', 'znga', 'zombie', 'zone',
       'zuck', 'zuckerberg'],
      dtype='object', length=2240)


In [93]:

#bow = bow.rename(columns = {'fit': 'fit_'})

#X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:,2:], bow.iloc[:,1])                                                  
#id_vals=X_test.index
y_train=bow["label"]
X_train=bow.drop({"ID","label"},axis=1)


In [94]:
"""CV_time = time.time()
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 
grid = GridSearchCV(xgb, params)
grid.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - CV_time))
"""

'CV_time = time.time()\n# A parameter grid for XGBoost\nparams = {\'min_child_weight\':[4,5], \'gamma\':[i/10.0 for i in range(3,6)],  \'subsample\':[i/10.0 for i in range(6,11)],\n\'colsample_bytree\':[i/10.0 for i in range(6,11)], \'max_depth\': [2,3,4]}\n\n# Initialize XGB and GridSearch\nxgb = XGBRegressor(nthread=-1) \ngrid = GridSearchCV(xgb, params)\ngrid.fit(X_train, y_train)\nprint("--- %s seconds ---" % (time.time() - CV_time))\n'

In [95]:
"""grid.best_params_"""

'grid.best_params_'

In [96]:
#Fitting XGB regressor with parameters obtained by Grid searchCV
params={'colsample_bytree': 1.0,
 'gamma': 0.3,
 'max_depth': 4,
 'min_child_weight': 4,
 'subsample': 0.8}
model_xgb = xgb.XGBRegressor(max_depth=4, min_child_weight= 4,gamma=0.3,subsample=0.8,colsample_bytree=1.0)

model_xgb.fit(X_train,y_train)
print (model_xgb)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1.0, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=4, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)


In [97]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(model_xgb.predict(X_train), y_train))
print("R2 score:" ,r2_score(model_xgb.predict(X_train),y_train))

mean squared error: 0.03527199670865589
R2 score: 0.4613759455472385


In [98]:
"""
X_test=bow_test.drop({"ID"},axis=1)
X_test = X_test.as_matrix()

y_test=data_test.processed_data["sentiment score"]
id_vals=y_test.index
"""

'\nX_test=bow_test.drop({"ID"},axis=1)\nX_test = X_test.as_matrix()\n\ny_test=data_test.processed_data["sentiment score"]\nid_vals=y_test.index\n'

In [99]:

#reorganize test columns as the train columns

#X_test = bow_test.as_matrix()

#X_test

bow_test=bow_test[bow.drop(["label"],axis=1).columns]
X_test=bow_test.drop({"ID"},axis=1)
y_test=data_test.processed_data["sentiment score"]
id_vals=X_test.index


In [100]:

print(len(bow_test.columns))
bow_test.head()
col=list(bow_test.columns)
col.index("bank")


2236


136

In [101]:
#if train_test_split used
"""tweet_row=[]
labels=["ID","tweet"]
for num in id_vals:
    current_row=[]
    current_id = bow.loc[num, "ID"]
                
    current_tweet=data.processed_data.loc[data.processed_data['id'] == current_id,"spans"].iloc[0]

                
    current_row.append(current_id)
    current_row.append(current_tweet)
    

   

    tweet_row.append(current_row) 
tweet_row"""

'tweet_row=[]\nlabels=["ID","tweet"]\nfor num in id_vals:\n    current_row=[]\n    current_id = bow.loc[num, "ID"]\n                \n    current_tweet=data.processed_data.loc[data.processed_data[\'id\'] == current_id,"spans"].iloc[0]\n\n                \n    current_row.append(current_id)\n    current_row.append(current_tweet)\n    \n\n   \n\n    tweet_row.append(current_row) \ntweet_row'

In [102]:
#Predict 
#output=grid.best_estimator_.predict(X_test)
tweet_row=list(data_test.processed_data["spans"])
old_pred=list(data_test.processed_data["sentiment score"])
output = model_xgb.predict(X_test)
#final_df = pd.DataFrame(tweet_row, columns=labels)
final_df = pd.DataFrame()
final_df["tweet"] = tweet_row

final_df["old_pred"] = old_pred

final_df["Prediction"] = output
final_df.to_csv("Output_1.csv",sep=",")
final_df.head(20)

,tweet,old_pred,Prediction
0,[Putting on a little $F short],-0.454,0.222071
1,[short some],-0.464,0.222071
2,[buying opportunity],0.445,0.278782
3,[Scaling Up on Long Position],0.661,0.379209
4,[its time to sell banks],-0.763,0.119029
5,[Entering long],0.627,0.379209
6,[picked some up],0.653,0.259054
7,"[time to accumulate for a long position, far m...",0.668,0.407423
8,"[Looking for a strong bounce, Lunchtime rally ...",0.460,0.293277
9,[Very intrigued with the technology and growth...,0.403,0.289607


In [103]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(output, y_test))
print("R2 score:" ,r2_score(output,y_test))

mean squared error: 0.24105996442903568
R2 score: -27.359540031512054
